In [43]:
using Gurobi
using QPSReader
using QuadraticModels
using NLPModels
using SolverTools
using SolverBenchmark
using DataFrames
using BenchmarkTools
using LinearAlgebra
using Plots

In [2]:
show_statuses()

STATUSES:
  :acceptable     => solved to within acceptable tolerances
  :exception      => unhandled exception
  :first_order    => first-order stationary
  :infeasible     => problem may be infeasible
  :max_eval       => maximum number of function evaluations
  :max_iter       => maximum iteration
  :max_time       => maximum elapsed time
  :neg_pred       => negative predicted reduction
  :not_desc       => not a descent direction
  :small_residual => small residual
  :small_step     => step too small
  :stalled        => stalled
  :unbounded      => objective function may be unbounded from below
  :unknown        => unknown
  :user           => user-requested stop


In [3]:
STATUSES

UndefVarError: UndefVarError: STATUSES not defined

In [87]:
##################################################################
####################### INFO ATTRIBUTES #########################
# https://www.gurobi.com/documentation/9.0/refman/attributes.html 

function createQuadraticModel(qpdata)
    # probleme du point initial
    #x0 = init_x0(qpdata.lvar, qpdata.uvar)
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
                        Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
                        lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
                        c0=qpdata.c0)

end

    

function OptimizeGurobi(QM)
    SM = SlackModel(QM)
    env = Gurobi.Env()
    
    # -1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 
    # 3=concurrent, 4=deterministic concurrent, 5=deterministic concurrent simplex.
    setparam!(env, "Method", 2)
    
    # set presolve to 0
    setparam!(env, "Presolve", 0)
    
    # no scaling
    setparam!(env, "ScaleFlag", 0)
    
    Aeq = jac(SM, SM.meta.x0)
    beq = SM.meta.lcon
    f = grad(SM, zeros(length(SM.meta.x0)))
    n,m = size(Aeq)
    model = gurobi_model(env; f = f,
                        Aeq = Aeq, beq = beq, 
                        lb = SM.meta.lvar, ub = SM.meta.uvar)
     # run optimization
    optimize(model)
    
    # y with dual: b'*y   s.t. A'*y <= c and y >= 0
    y = zeros(n)
    for i=1:n
        y[i] = Gurobi.get_dblattrelement(model, "Pi", i)
    end
    
    s = zeros(m) # s_l - s_u
    for i=1:m
        s[i] = Gurobi.get_dblattrelement(model, "RC", i)
    end
    
    # outputs
    optim_info = get_optiminfo(model)
    if optim_info.status == :optimal
        status = :acceptable
    elseif optim_info.status == :iteration_limit
        status = :max_iter
    elseif optim_info.status == :unbounded 
        status = :unbounded 
    else
        status = :unknown
    end
    
    x = get_solution(model)
    stats = GenericExecutionStats(status, SM, solution = x,
                                  objective = get_objval(model), iter = optim_info.iter_count,
                                  primal_feas = norm(Aeq * x - beq),
                                  dual_feas = norm(Aeq' * y + s - f),
                                  multipliers = y,
                                  elapsed_time = optim_info.runtime)

    return stats
end

function OptimizeGurobi(qpdata::QPSData)
    return OptimizeGurobi(createQuadraticModel(qpdata))
end



OptimizeGurobi (generic function with 2 methods)

In [88]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib"
pb2 = string(path_pb, "\\AFIRO.SIF")
qpdata2 = readqps(pb2);

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [89]:
stats2 = OptimizeGurobi(qpdata2)
println(stats2)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 27 rows, 51 columns and 102 nonzeros
Model fingerprint: 0x84b9128a
Coefficient statistics:
  Matrix range     [1e-01, 2e+00]
  Objective range  [3e-01, 1e+01]
  Bounds range     [8e+01, 5e+02]
  RHS range        [4e+01, 4e+01]
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 6.300e+01
 Factor NZ  : 2.180e+02
 Factor Ops : 2.146e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.13189124e+03  0.00000000e+00  6.25e+03 6.00e-01  1.37e+03     0s
   1  -9.99933753e+01 -4.18167525e+03  6.00e+02 1.33e-15  1.72e+02     0s
   2  -1.69965259e+02 -1.11383081e+03  4.26e-13 1.33e-15  1.85e+01     0s
   3  -3.86699787e+02 -5.12104722e+02  8.47e-11 1.11e-15  2.46e+00     0s
   4  -4.62718474e+02 -4.72312092e+02  2.84e-12 6.48e-16  1.8

In [60]:
# problem 3   kb2    obj  -1.7499001299E+03
kb2 = string(path_pb, "\\KB2.SIF")
qpdata3 = readqps(kb2)
stats3 =  OptimizeGurobi(qpdata3)
println(stats3)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 43 rows, 68 columns and 313 nonzeros
Model fingerprint: 0xd1d84034
Coefficient statistics:
  Matrix range     [2e-01, 1e+02]
  Objective range  [9e-02, 2e+01]
  Bounds range     [5e+00, 2e+02]
  RHS range        [0e+00, 0e+00]
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 4.020e+02
 Factor NZ  : 5.620e+02
 Factor Ops : 8.522e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.58276378e+02 -1.82748880e+03  2.05e+02 6.04e-01  4.68e+02     0s
   1  -1.96669587e+02 -1.09754794e+03  7.18e+01 5.17e+00  1.80e+02     0s
   2  -4.25373491e+02 -5.37892865e+02  4.52e+00 1.49e+00  4.30e+01     0s
   3  -9.27892694e+02 -2.48425362e+02  2.74e-11 4.61e-01  1.30e+01     0s
   4  -1.36836748e+03 -2.60199720e+02  9.99e-11 3.07e-01  6.4

┌ Info: Using 'KB2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'FAT7..J.' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '77BOUND' as BOUNDS (l. 227)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622


  iterations: 11
  elapsed time: 0.007917404174804688


# Test sur tous les pb

In [7]:
df_test = DataFrame(id = Int[],
         name = String[],
         status = Symbol[],
         objective = Float64[],
         time = Float64[],
         iter = Int[],
         primal_feas = Float64[],
         dual_feas = Float64[])


i = 0
for file_name in readdir(path_pb)
    if file_name[end-3:end] == ".SIF" && !(file_name in ["80BAU3B.SIF" ; "BORE3D.SIF"; 
                                                        "CAPRI.SIF"; "CZPROB.SIF"; 
                                                        "ETAMACRO.SIF"; "FINNIS.SIF";
                                                        "FORPLAN.SIF"; "GREENBEA.SIF";
                                                        "GREENBEB.SIF"; "MAROS.SIF";
                                                        "NESM.SIF"; "PEROLD.SIF";
                                                         "PILOT-JA.SIF"; "PILOT-WE.SIF";
                                                         "PILOT.SIF"; "PILOT4.SIF";
                                                         "PILOT87.SIF"; "PILOTNOV.SIF";
                                                          "RECIPELP.SIF"; "SHELL.SIF";
                                                         "SIERRA.SIF"; "STAIR.SIF";
                                                         "STANDATA.SIF"; "STANDGUB.SIF";
                                                        "STANDMPS.SIF"; "TUFF.SIF";
                                                        "VTP-BASE.SIF"]) # problems with fixed variables
        
        i += 1
        println(file_name)
        pb_i = string(path_pb, "\\", file_name)
        if file_name in ["BLEND.SIF"; "DFL001.SIF"; "FORPLAN.SIF"; "GFRD-PNC.SIF"; "SIERRA.SIF"]
            qpdata_i = readqps(pb_i, mpsformat=:fixed) 
        else
            qpdata_i = readqps(pb_i) 
        end
        SM_i = SlackModel(createQuadraticModel(qpdata_i))
        if SM_i.meta.ifix != []
            println("problem fixed variables ", SM_i.meta.ifix, "   ", file_name)
        end
        stats_i = OptimizeGurobi(qpdata_i)
        
        push!(df_test, Dict(:id => i,
                :name => file_name[1:end-4] ,
                             :status => stats_i.status,
                             :objective => stats_i.objective,
                             :time => stats_i.elapsed_time,
                             :iter => stats_i.iter,
                             :primal_feas => stats_i.primal_feas,
                             :dual_feas => stats_i.dual_feas))

    end
    if i == 15
        break
        break
    end
end

25FV47.SIF


┌ Info: Using '25FV47' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'R0000' as objective (l. 21)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350


Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 821 rows, 1876 columns and 10705 nonzeros
Model fingerprint: 0x5534e7d0
Coefficient statistics:
  Matrix range     [2e-04, 2e+02]
  Objective range  [1e-03, 1e+02]
  Bounds range     [1e+00, 2e+03]
  RHS range        [2e-01, 6e+02]
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.107e+04
 Factor NZ  : 3.891e+04 (roughly 1 MByte of memory)
 Factor Ops : 2.797e+06 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.52038651e+05  0.00000000e+00  4.91e+04 1.60e+01  2.76e+04     0s
   1   1.72788987e+05 -4.60949622e+05  2.49e+04 2.75e+01  1.38e+04     0s
   2   6.93512108e+04 -1.01291183e+06  8.15e+03 5.25e+00  4.85e+03     0s
   3   2.34667431e+04 -6.76015978e+05  1.51e+03 3.33e-01  1.01e+03     0s
   4   1.38943866e+04 -3.083

┌ Info: Using '.00001' as RHS (l. 6793)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'ADLITTLE' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using '.Z....' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'ZZZZ0001' as RHS (l. 333)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AGG' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\pack

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 488 rows, 615 columns and 2862 nonzeros
Model fingerprint: 0xee52fdcb
Coefficient statistics:
  Matrix range     [2e-05, 4e+02]
  Objective range  [3e+00, 1e+02]
  Bounds range     [1e+02, 6e+06]
  RHS range        [0e+00, 0e+00]
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.118e+04
 Factor NZ  : 1.736e+04 (roughly 1 MByte of memory)
 Factor Ops : 7.034e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.52596660e+09  0.00000000e+00  1.12e+06 3.14e+02  1.34e+08     0s
   1   1.37103623e+09 -1.84057298e+09  5.60e+05 1.21e+02  5.97e+07     0s
   2  -2.96512251e+06 -1.13105100e+09  1.59e+04 6.28e-13  3.26e+06     0s
   3  -3.86666326e+07 -3.31485286e+08  6.16e+03 1.36e-12  9.96e+05     0s
   4  -4.77418304e+07 -1.57923

...

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 351 rows, 726 columns and 3827 nonzeros
Model fingerprint: 0xd61e9810
Coefficient statistics:
  Matrix range     [1e-02, 3e+03]
  Objective range  [1e-02, 4e+01]
  Bounds range     [1e+00, 3e+03]
  RHS range        [1e+01, 2e+01]
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 5.077e+03
 Factor NZ  : 9.357e+03 (roughly 1 MByte of memory)
 Factor Ops : 2.791e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.99822366e+04 -2.46465518e+04  4.50e+04 3.34e-01  9.86e+02     0s
   1   9.13362017e+03 -2.38872811e+04  1.38e+04 1.24e+00  3.68e+02     0s
   2   4.36130822e+03 -2.33482584e+04  3.88e+03 7.91e-01  1.43e+02     0s
   3   2.86157363e+03 -1.35120934e+04  2.12e+03 8.59e-02  5.26e+01     0s
   4   7.16413554e+02 -5.31493

┌ Info: Using 'RANGE1' as RANGES (l. 2446)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:552
┌ Info: Using 'INTBOU' as BOUNDS (l. 2492)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622
┌ Info: Using 'BOEING2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 23)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350


Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 166 rows, 305 columns and 1358 nonzeros
Model fingerprint: 0x9f508ec0
Coefficient statistics:
  Matrix range     [1e-02, 3e+03]
  Objective range  [1e-02, 7e+00]
  Bounds range     [1e+00, 1e+05]
  RHS range        [3e+01, 5e+01]
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.876e+03
 Factor NZ  : 3.635e+03
 Factor Ops : 1.043e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.83866930e+04 -2.30750873e+03  5.11e+04 5.38e-01  3.20e+03     0s
   1   1.21398620e+04 -4.28235138e+03  1.50e+04 1.26e+00  1.07e+03     0s
   2   3.60577213e+03 -1.11917747e+04  5.82e+03 4.73e-02  3.82e+02     0s
   3   3.55784679e+02 -7.10143215e+03  5.20e+02 3.23e-14  4.60e+01     0s
   4   9.45895148e+01 -2.98287468e+03  1.56e+02 4.88e-15  

┌ Info: Using 'RHS1' as RHS (l. 898)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'RANGE1' as RANGES (l. 919)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:552
┌ Info: Using 'INTBOU' as BOUNDS (l. 930)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622
┌ Info: Using 'BRANDY' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using '10000A' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'ZZZZ0001' as RHS (l. 1378)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'CRE-A' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 3542)
└ @ QPSReader C:\Users\Geoffroy Lecon

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 3516 rows, 7248 columns and 18168 nonzeros
Model fingerprint: 0x0071ac78
Coefficient statistics:
  Matrix range     [6e-01, 7e+01]
  Objective range  [2e+02, 3e+05]
  Bounds range     [1e+00, 3e+03]
  RHS range        [0e+00, 0e+00]
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.075e+04
 Factor NZ  : 5.278e+04 (roughly 5 MBytes of memory)
 Factor Ops : 1.218e+06 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.11133920e+08  0.00000000e+00  3.01e+03 0.00e+00  2.13e+05     0s
   1   2.37896109e+08 -1.20435766e+07  1.66e+03 1.98e+03  1.19e+05     0s
   2   7.63261403e+07 -1.31557582e+07  3.31e+02 3.78e+02  2.78e+04     0s
   3   5.19200066e+07  3.70724493e+06  1.53e+02 1.06e+02  1.24e+04     0s
   4   4.08344286e+07  1.2

In [8]:
markdown_table(stdout, df_test)

|    id |     name |     status | objective |     time |  iter | primal_feas | ⋯
|-------|----------|------------|-----------|----------|-------|-------------| ⋯
|     1 |   25FV47 | acceptable |   5.5e+03 |  4.7e-02 |    99 |     4.0e-12 | ⋯
|     2 | ADLITTLE | acceptable |   2.3e+05 |  6.0e-03 |    31 |     2.3e-13 | ⋯
|     3 |    AFIRO | acceptable |  -4.6e+02 |  9.9e-04 |    12 |     1.8e-14 | ⋯
|     4 |      AGG | acceptable |  -3.6e+07 |  2.4e-02 |    66 |     4.7e-10 | ⋯
|     5 |     AGG2 | acceptable |  -2.0e+07 |  1.9e-02 |    54 |     1.2e-10 | ⋯
|     6 |     AGG3 | acceptable |   1.0e+07 |  2.3e-02 |    52 |     1.2e-10 | ⋯
|     7 |    BANDM | acceptable |  -1.6e+02 |  1.1e-02 |    21 |     2.2e-13 | ⋯
|     8 | BEACONFD | acceptable |   3.4e+04 |  6.0e-03 |    80 |     8.1e-12 | ⋯
|     9 |    BLEND | acceptable |  -3.1e+01 |  1.9e-02 |    14 |     1.4e-14 | ⋯
|    10 |     BNL1 | acceptable |   2.0e+03 |  5.1e-02 |   352 |     3.4e-13 | ⋯
|    11 |     BNL2 | accepta

In [11]:
save_path = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\results"

"C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\results"

In [14]:

open(string(save_path, "\\alpha.tex"), "w") do io
  println(io, "\\documentclass[varwidth=20cm,crop=true]{standalone}")
  println(io, "\\usepackage{longtable}")
  println(io, "\\begin{document}")
  latex_table(io, df_test)
  println(io, "\\end{document}")
end

In [15]:
stats = Dict(:gurobi => df_test)
stats[:gurobi]

,id,name,status,objective,time,iter,primal_feas,dual_feas
,Int64,String,Symbol,Float64,Float64,Int64,Float64,Float64
1,1,25FV47,acceptable,5501.85,0.0468559,99,4.00746e-12,8.52651e-14
2,2,ADLITTLE,acceptable,225495.0,0.00597763,31,2.27374e-13,4.54747e-13
3,3,AFIRO,acceptable,-464.753,0.000991821,12,1.77636e-14,0.0
4,4,AGG,acceptable,-3.59918e7,0.0239353,66,4.65661e-10,9.09495e-13
5,5,AGG2,acceptable,-2.02393e7,0.0189514,54,1.16415e-10,1.42109e-14
6,6,AGG3,acceptable,1.03121e7,0.0229759,52,1.16415e-10,1.42109e-14
7,7,BANDM,acceptable,-158.628,0.0110054,21,2.22489e-13,1.08524e-14
8,8,BEACONFD,acceptable,33592.5,0.00598526,80,8.06466e-12,1.38778e-15
9,9,BLEND,acceptable,-30.8121,0.0187912,14,1.42109e-14,1.77636e-15


In [16]:
using JLD2, FileIO
save(string(save_path, "\\test_df.jld2"), stats)

Error encountered while saving "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\results\\test_df.jld2".

Fatal error:


ArgumentError: ArgumentError: keys must be strings (the names of variables), got gurobi

In [ ]:
load(string(save_path, "\\test_df.jld2"))

In [105]:
for file_name in readdir(path_pb)
    println("problem ", file_name)
    
end

problem 25FV47.SIF
problem 80BAU3B.SIF
problem ADLITTLE.SIF
problem AFIRO.SIF
problem AGG.SIF
problem AGG2.SIF
problem AGG3.SIF
problem BANDM.SIF
problem BEACONFD.SIF
problem BLEND.SIF
problem BNL1.SIF
problem BNL2.SIF
problem BOEING1.SIF
problem BOEING2.SIF
problem BORE3D.SIF
problem BRANDY.SIF
problem CAPRI.SIF
problem CRE-A.SIF
problem CRE-B.SIF
problem CRE-C.SIF
problem CRE-D.SIF
problem CYCLE.SIF
problem CZPROB.SIF
problem D2Q06C.SIF
problem D6CUBE.SIF
problem DEGEN2.SIF
problem DEGEN3.SIF
problem DFL001.SIF
problem E226.SIF
problem ETAMACRO.SIF
problem FFFFF800.SIF
problem FINNIS.SIF
problem FIT1D.SIF
problem FIT1P.SIF
problem FIT2D.SIF
problem FIT2P.SIF
problem FORPLAN.SIF
problem GANGES.SIF
problem GFRD-PNC.SIF
problem GREENBEA.SIF
problem GREENBEB.SIF
problem GROW15.SIF
problem GROW22.SIF
problem GROW7.SIF
problem ISRAEL.SIF
problem KB2.SIF
problem KEN-07.SIF
problem KEN-11.SIF
problem KEN-13.SIF
problem KEN-18.SIF
problem LOTFI.SIF
problem MAROS-R7.SIF
problem MAROS.SIF
probl

In [16]:
T = typeof(SM1.meta)
for (name, typ) in zip(fieldnames(T), T.types)
    println("type of the fieldname $name is $typ")
end

type of the fieldname nvar is Int64
type of the fieldname x0 is Array{T,1} where T
type of the fieldname lvar is Array{T,1} where T
type of the fieldname uvar is Array{T,1} where T
type of the fieldname ifix is Array{Int64,1}
type of the fieldname ilow is Array{Int64,1}
type of the fieldname iupp is Array{Int64,1}
type of the fieldname irng is Array{Int64,1}
type of the fieldname ifree is Array{Int64,1}
type of the fieldname iinf is Array{Int64,1}
type of the fieldname nbv is Int64
type of the fieldname niv is Int64
type of the fieldname nlvb is Int64
type of the fieldname nlvo is Int64
type of the fieldname nlvc is Int64
type of the fieldname nlvbi is Int64
type of the fieldname nlvci is Int64
type of the fieldname nlvoi is Int64
type of the fieldname nwv is Int64
type of the fieldname ncon is Int64
type of the fieldname y0 is Array{T,1} where T
type of the fieldname lcon is Array{T,1} where T
type of the fieldname ucon is Array{T,1} where T
type of the fieldname jfix is Array{Int64,1